#### Evolution Node 
- Task, Input, Output
- LLM guided evolution
- LLM-engineered Code Function or Prompt

In [1]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.eoh_evolution import EvolNode


# Code + Compilor Task
mp = MetaPrompt("Generate Fibonacci sequence.", "fibonacci", ["n"], ["sequence"], ["int"], ["list"], PromptMode.CODE) # 
node = EvolNode(mp, None, None)
reasoning, code = node._evolve("i1", replace=True)
input_dict = {"n": 10}
# node(input_dict) # Ok we need a output dictionary here as well ...


# Prompt + LLM Task
# mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name_of_celebrity"], ["age"], PromptMode.PROMPT) # 
# node = EvolNode(mp, None, None)
# reasoning, code = node._evolve("i1", replace=True)
# node("Deilireba")

Could not load vllm class, check CUDA support and GPU RAM size


In [2]:
print(code)

def fibonacci(n: int) -> list:
    if n <= 0:
        return []
    if n == 1:
        return [0]
    sequence = [0, 1]
    while len(sequence) < n:
        sequence.append(sequence[-1] + sequence[-2])
    return sequence


In [3]:
node(input_dict) # Ok we need a output dictionary here as well ...

{'sequence': [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]}

In [5]:
print(code)
code = code.replace("import List", "from typing import List")

import List

def fibonacci(n: int) -> List[int]:
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    elif n == 2:
        return [0, 1]

    sequence = [0, 1]
    for i in range(2, n):
        sequence.append(sequence[-1] + sequence[-2])
    return sequence


'from typing import List\n\ndef fibonacci(n: int) -> List[int]:\n    if n <= 0:\n        return []\n    elif n == 1:\n        return [0]\n    elif n == 2:\n        return [0, 1]\n\n    sequence = [0, 1]\n    for i in range(2, n):\n        sequence.append(sequence[-1] + sequence[-2])\n    return sequence'

In [2]:
eval_prompt = mp._get_eval_prompt()
print(eval_prompt)

AttributeError: 'MetaPrompt' object has no attribute 'input_types'

In [5]:
print(code)

from methods.meta_execute import compile_check
compile_check(code)

compile(code, '<string>', 'exec')

import List

def fibonacci(n: int) -> List[int]:
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    elif n == 2:
        return [0, 1]
    
    sequence = [0, 1]
    for i in range(2, n):
        next_value = sequence[-1] + sequence[-2]
        sequence.append(next_value)
    
    return sequence


<code object <module> at 0x2abe81200, file "<string>", line 1>

In [6]:
def safe_exec(code_str, input_dict):
    local_vars = {}
    exec(code_str, {}, local_vars)
    if 'fibonacci' in local_vars and callable(local_vars['fibonacci']):
        return local_vars['fibonacci'](input_dict['n'])
    else:
        raise ValueError("Function 'fibonacci' not found or not callable")

# Test the function with sample input
input_dict = {"n": 10}
try:
    result = safe_exec(code, input_dict)
    print(f"Execution successful! Result for n={input_dict['n']}: {result}")
except Exception as e:
    print(f"Execution failed: {str(e)}")

Execution failed: No module named 'List'


In [4]:
from typing import List

In [2]:
print(code)

import List

def fibonacci(n: int) -> List[int]:
    sequence = []
    if n <= 0:
        return sequence
    elif n == 1:
        return [0]
    
    sequence = [0, 1]
    while len(sequence) < n:
        sequence.append(sequence[-1] + sequence[-2])
    
    return sequence


#### Evolution Graph
- Take care of connecting each components (unfolding dictionary object etc.)
- Planning topology with evolution node

In [2]:
from methods.meta_prompt import MetaPlan, extract_json_from_text

mp = MetaPlan("Help me date Dilireba, I'm a white colar worker living on salary.")
prompt = mp._get_prompt_i1()
print("MetaPrompt I1: ")
print(prompt)

from methods.llm import get_openai_response
response = get_openai_response(prompt)

MetaPrompt I1: 
Goal: Help me date Dilireba, I'm a white colar worker living on salary.
First, describe the intuition for your tactics and main steps in one sentence. The description must be inside a brace.
Generate a JSON-style plan represented as a Directed Acyclic Graph (DAG) to achieve the goal. Use creative topology in the DAG, include parallel tasks if required.

The plan should include:

- **Nodes**: Each node represents a key action or step and must contain the following attributes:
  - `task`: Description of the task.
  - `name`: Concise name used for the task function.
  - `input`: The resources, information, or prerequisites needed to perform the action.
  - `output`: The immediate result or outcome of the action.
  - `target`: The purpose or goal that the action contributes to.
  - `mode`: The execution mode for this task ("CODE" or "PROMPT").

- **Edges**: Each edge represents a dependency or relationship between nodes, indicating that one step supports or leads to another

In [11]:
import re 
# from methods.meta_prompt import parse_json_from_response

tactic = re.findall(r"\{(.*)\}", response, re.DOTALL)
plan_dict = extract_json_from_text(response)

In [5]:
from methods.meta_prompt import MetaPrompt, MetaPlan, extract_json_from_text
from methods.eoh_evolution import EvolNode
from methods.llm import get_openai_response as get_response
from typing import Optional, List, Dict

# Collect MetaPrompt from parsed Plan-Graph Response
meta_node_prompts = []
for node in plan_dict["nodes"]:
    node_prompt = MetaPrompt(task=node.get("task"),  func_name=node.get("name"), input=node.get("input"), output=node.get("output"), mode=node.get("mode").lower())
    meta_node_prompts.append(node_prompt)
    
edges = plan_dict["edges"]

EvolNode(meta_prompt = node_prompt)


#### EvalNode Test: 
At the beginning level, generation of EvalNode (when code is used here) should compile with success, and it should takes input and produce output. 